In [1]:
import findspark
findspark.init()
import pyspark
from pyspark import SparkContext
import numpy as np
import time

# Variables globales
g_numf = g_numc = 0 # Número de filas, número de columnas

In [2]:
t1 = time.time()
sc    = SparkContext("local[*]", "MediaVarianza_3")
datos = sc.textFile("../../../datos/data_ok.csv")
g_fil = datos.count() # 5000
g_col = len(datos.take(1)[0].split(" ")) # 211
t2 = time.time()
print(g_fil, g_col)
print("Tiempo transcurrido: {} segundos.".format(t2 - t1))

5000 211
Tiempo transcurrido: 3.266185760498047 segundos.


In [3]:
# fila: 211 flotantes separados por espacios (cada flotante es una columna)
# Los convertimos a un array de tuplas (j, v)
#  j: nº de columna
#  v: valor (flotante)
def formato_fila(fila):
    lista = []
    fila = fila.split(" ")
    for i in range(len(fila)):
        lista.append((i, float(fila[i])))
    return lista

t1 = time.time()
datos = datos.flatMap(formato_fila) # 1055000 tuplas (columna, valor)
t2 = time.time()
print("Tiempo transcurrido: {} segundos.".format(t2 - t1))

Tiempo transcurrido: 5.91278076171875e-05 segundos.


In [4]:
# datos: RDD de (1 millón de) tuplas (j, v)
#  j: columna
#  v: valor
# Devuelve un RDD de (211) tuplas (j, m)
#  j: columna
#  m: media de la columna
def calcular_medias(datos):
    sumas  = datos.reduceByKey(lambda x, y: x + y) # 211 tuplas (columna, suma)
    medias = sumas.map(lambda x: (x[0], x[1]/g_fil)) # 211 tuplas (columna, media)
    return medias
t1 = time.time()
medias = calcular_medias(datos) # 211 tuplas (columna, media)
t2 = time.time()
print("Tiempo transcurrido: {} segundos.".format(t2 - t1))

Tiempo transcurrido: 0.03116321563720703 segundos.


In [5]:
# datos: RDD de (1 millón de) tuplas (j, v)
#  j: columna
#  v: valor
# Devuelve un RDD de (211) tuplas (j, E[x]², E[x²])
#  j: columna
#  E[x]²: Cuadrado de la media de la variable X
#  E[x²]: Momento no centrado de orden 2 (media de los cuadrados)
def ambas_medias(datos):
    datos2 = datos.map(lambda x: (x[0], (x[1], x[1]**2))) # (j, (v, v²))
    sumas  = datos2.reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))
    medias = sumas.map(lambda x:  (x[0], (x[1][0]/g_fil, x[1][1]/g_fil)))
    medias = medias.map(lambda x: (x[0], (x[1][0]**2, x[1][1])))
    return medias

# medias: RDD de (211) tuplas (j, E[x]², E[x²])
# Devuelve un RDD de (211) tuplas (j, d)
#  j: columna
#  d: desviación estándar
# Varianza = E[x²] - E[x]²
def formato_desvst(medias): #           j     E[x²]      E[x]²
    varianzas = medias.map(lambda x: (x[0], x[1][1] - x[1][0]))
    desviests = varianzas.map(lambda x: (x[0], x[1]**0.5)) # desviest = sqrt(varianza)
    return desviests

t1 = time.time()
ambasmedias = ambas_medias(datos)
desviests   = formato_desvst(ambasmedias)
t2 = time.time()

print("Tiempo transcurrido: {} segundos.".format(t2 - t1))

Tiempo transcurrido: 0.01779460906982422 segundos.


In [6]:
todojunto = medias.join(desviests).sortByKey().collect()

for i in todojunto:
    print("Columna %d\n\tMedia: %0.3f\n\tStdev: %0.3f"%(i[0]+1, i[1][0], i[1][1]))

Columna 1
	Media: 427.838
	Stdev: 633.891
Columna 2
	Media: 8544.564
	Stdev: 56288.306
Columna 3
	Media: 1006.771
	Stdev: 1720.350
Columna 4
	Media: 1409608.915
	Stdev: 2425645.280
Columna 5
	Media: 1.000
	Stdev: 0.000
Columna 6
	Media: 6.654
	Stdev: 1.517
Columna 7
	Media: 1460.000
	Stdev: 0.000
Columna 8
	Media: 762.313
	Stdev: 559.761
Columna 9
	Media: 140.239
	Stdev: 140.813
Columna 10
	Media: 506626.920
	Stdev: 587803.754
Columna 11
	Media: 28973.706
	Stdev: 2552.306
Columna 12
	Media: 0.011
	Stdev: 0.180
Columna 13
	Media: 0.946
	Stdev: 0.226
Columna 14
	Media: 7.501
	Stdev: 1.989
Columna 15
	Media: 1411.073
	Stdev: 120.962
Columna 16
	Media: 3307.134
	Stdev: 4436.947
Columna 17
	Media: 198.772
	Stdev: 322.426
Columna 18
	Media: 835825.356
	Stdev: 3515411.545
Columna 19
	Media: 43811.783
	Stdev: 20858.306
Columna 20
	Media: 0.000
	Stdev: 0.000
Columna 21
	Media: 238.935
	Stdev: 337.770
Columna 22
	Media: 224802.519
	Stdev: 389591.731
Columna 23
	Media: 16.719
	Stdev: 55.770
Colum